# Setup Notebook

## Install Dependencies

In [1]:
! pip install simpletransformers

     |████████████████████████████████| 249 kB 582 kB/s            
     |████████████████████████████████| 43 kB 1.5 MB/s             
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 10.1 MB 6.9 MB/s            
     |████████████████████████████████| 4.3 MB 35.9 MB/s            
     |████████████████████████████████| 76 kB 3.8 MB/s             
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=8b0231e71ee3e3b240622a55fe4b9a488d48aa064f9dbd21d9efcf5fcfdb3607
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


## Load Imports

In [2]:
import numpy as np
import pandas as pd
import sklearn

from pathlib import Path
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs

# Configuration File

In [3]:
config = {'data': {'subset': 1,
                   'split': 1,
                   'columns': ['Headline', 'articleBody', 'Stance'],
                  #  'columns': ['Headline', 'articleBody', 'related'],
                  },
          
          'model': {
#                     'model_type': 'roberta',
#                     'model_name': 'roberta-base', #roberta-large
                    'model_type': 'bert',
                    'model_name': 'bert-base-uncased',
                   },
    
          'training':{'learning_rate':1e-5,
                      'num_train_epochs': 1,
                      'train_batch_size': 4,
                      'eval_batch_size': 4,
                      'sliding_window': True,
                      'save_steps': -1,
                      }
}

In [4]:
path = Path('../input/fnc-1/')

! ls {path}

example_test.txt  example_train.txt  test.csv  test.txt  train.csv  train.txt


# Load Data

In [5]:
def read_data(path: str, name: str):
    '''Reads csv file
    
    Args:
        path (str): parent directory to file
        name (str): type of csv to load (train or test)
        
    Returns:
        pandas.core.frame.DataFrame of joined bodies and stances        
    '''
    df = pd.read_csv(f'{path}/{name}.csv', encoding='utf-8')
    return df

In [6]:
def load_data(path: str, name: str, config: dict):
    '''Read and process csv to desired format
    
    Args:
        path (str): parent directory to file
        name (str): type of csv to load (train or test)
        config (dict): configuration of data loading parameters
    
    Returns:
        Preprocessed data
    '''
    df = read_data(path, name)
    processed_df = df[config['columns']]
    
    text = np.append(processed_df['Headline'].values, processed_df['articleBody'].values)

    return text

In [7]:
# train = load_data(path, 'train', config['data'])
# test = load_data(path, 'test', config['data'])

# Save File (Run Once)

In [8]:
# with open('train.txt', 'w') as f:
#     for item in train:
#         f.write("%s\n" % item)

In [9]:
# with open('test.txt', 'w') as f:
#     for item in test:
#         f.write("%s\n" % item)

# Train Model

In [10]:
model = LanguageModelingModel(config['model']['model_type'], config['model']['model_name'], args=config['training'])

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
train_file = f'{path}/train.txt'
test_file = f'{path}/test.txt'

In [12]:
model.train_model(train_file)

  0%|          | 0/648593 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/164819 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/simpletransformers/language_modeling/language_modeling_model.py:845: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", User

(164819, 1.3241955318489946)

# Evaluate Model

In [13]:
result = model.eval_model(test_file)
result

  0%|          | 0/312564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79526 [00:00<?, ?it/s]

{'eval_loss': 2.510800224607348, 'perplexity': tensor(12.3148)}

In [14]:
model.save_model()